# Indexing for Faster Queries
**ApertureDB relies on [indexes](https://docs.aperturedata.io/query_language/Reference/db_commands/CreateIndex) to enable faster queries on attributes of interest or embeddings.**

You can create an index before adding any object or connection (fastest).

You can add an index after the corresponding objects or connections already exist in the database (slower since it has to build the index)

In [1]:
# Set up machine and login information to use in the rest of this notebook
from getpass import getpass

dbinstance = input("Machine name: ")
password = getpass("Password:")

Machine name:  demo.gcp.cloud.aperturedata.io
Password: ········


In [2]:
# Connector class offers a way to connect to an instance of ApertureDB
from aperturedb import Connector

client = Connector.Connector(dbinstance, user="admin", password=password)

## Create an index on Ingredient's name

If we often search for an ingredient by its name, it should be indexed

In [3]:
query = [{
    "CreateIndex": {
        "class": "Ingredient",
        "index_type": "entity",
        "property_key": "name"
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "CreateIndex": {
            "status": 0
        }
    }
]


### Verify index using GetSchema

In [4]:
# CHECK if indexed parameter for Ingredient's name is True
# https://docs.aperturedata.io/query_language/Reference/db_commands/GetSchema
query = [{
    "GetSchema": {
        "type": "entities"
    }
}]

# Execute the query to get back a JSON response for GetStatus 
response, blobs = client.query(query)

print(response[0]["GetSchema"]["entities"]["classes"]["Ingredient"]["properties"]["name"])

[1, True, 'String']


## Remove the index

Indexes occupy space - we can remove them when unnecessary

In [5]:
query = [{
    "RemoveIndex": {
        "class": "Ingredient",
        "index_type": "entity",
        "property_key": "name"
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "RemoveIndex": {
            "status": 0
        }
    }
]


### Verify index using GetSchema

Here we check whether the `name` property of `Ingredient` entities is indexed.  See [GetSchema](https://docs.aperturedata.io/query_language/Reference/db_commands/GetSchema).  We're expecting it not to be, because we have not created an index yet.

In [6]:
query = [{
    "GetSchema": {
        "type": "entities"
    }
}]

# Execute the query to get back a JSON response for GetStatus 
response, blobs = client.query(query)

print(response[0]["GetSchema"]["entities"]["classes"]["Ingredient"]["properties"]["name"])

[1, False, 'String']


## Using Python SDK Utils for Indexing

The [Utils package](https://docs.aperturedata.io/python_sdk/helpers/Utils#create_entity_index) in ApertureDB Python SDK provides a lot of helper functions. One of the things they can help you to do is to add indexes.

In [7]:
from aperturedb.Utils import Utils

utils = Utils(client)
utils.create_entity_index(class_name="Ingredient", property_key="name")

True

Check again whether the `name` property of `Ingredient` entities is now indexed. 
We're expecting that it will be, because we just created an index.

In [8]:
query = [{
    "GetSchema": {
        "type": "entities"
    }
}]

# Execute the query to get back a JSON response for GetStatus 
response, blobs = client.query(query)

print(response[0]["GetSchema"]["entities"]["classes"]["Ingredient"]["properties"]["name"])

[1, True, 'String']


The Utils module can also help us to remove an index.

In [9]:
utils.remove_entity_index(class_name="Ingredient", property_key="name")

True

Now check again whether the `name` property of `Ingredient` entities is indexed.  We're expecting that it won't be, because we just removed the index.

In [10]:
query = [{
    "GetSchema": {
        "type": "entities"
    }
}]

# Execute the query to get back a JSON response for GetStatus 
response, blobs = client.query(query)

print(response[0]["GetSchema"]["entities"]["classes"]["Ingredient"]["properties"]["name"])

[1, False, 'String']
